In [1]:
!pip install fake_useragent
from bs4 import BeautifulSoup
from bs4.element import Tag
import requests
import time
from fake_useragent import UserAgent
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

In [4]:
# скачиваю данные в DataFrame из файла
df_raw = pd.read_csv('parsing data.csv')
df_raw.drop(columns = ['Unnamed: 0'], inplace = True)
df_raw.head(2)

,Личность подтверждена,Статус,Стаж,На сайте,Заказы в работе,Оценка,Количество отзывов,Формат обучения,Образование,Предмет,Цена,С отличием
0,False,Частный преподаватель,7,6.0,107.0,5.00,51.0,Дистанционно,МГУ им. М.В. Ломоносова,Русский язык,2112.5,False
1,True,Школьный преподаватель,38,6.0,139.0,4.89,41.0,Дистанционно,Волгоградский государственный педагогический ...,Русский язык,1000.0,False


Заполненяем пропуски:
1. Наны в признаках: `На сайте, Заказы в работе, количество отзывов` заполняем нулями
2. Оценку для преподавателей, у которой ее нет по каким-то причинам, заполняем средним значением
3. Для преподавателей, не указавших статус или формат обучения выбираем наиболее часто встречавшееся значение среди других тьюторов.
4. Пару анкет было без цены - мы их удалили

In [5]:
# Заполнение пропусков в столбце 'на сайте' значением 0
df_raw['На сайте'] = df_raw['На сайте'].fillna(0)

# Заполнение пропусков в столбце 'Заказы в работе' значением 0
df_raw['Заказы в работе'] = df_raw['Заказы в работе'].fillna(0)

# Заполнение пропусков в столбце 'Количество отзывов' значением 0
df_raw['Количество отзывов'] = df_raw['Количество отзывов'].fillna(0)

# Заполнение пропусков в столбце 'Оценка' средним значением
mean_mark = df_raw['Оценка'].astype(float).mean()
df_raw['Оценка'] = df_raw['Оценка'].fillna(mean_mark)

# Заполнение пропусков в столбце 'Статус' самым популярным значением
most_common_stat = df_raw['Статус'].mode()
df_raw['Статус'] = df_raw['Статус'].fillna(most_common_stat)

# Заполнение пропусков в столбце 'Формат обучения' самым популярным значением
most_common_format = df_raw['Формат обучения'].mode()
df_raw['Формат обучения'] = df_raw['Формат обучения'].fillna(most_common_format)

# удаляем анкету без цены
df_raw = df_raw[df_raw['Цена'].notna()]

In [9]:
df_raw.isna().sum()

Личность подтверждена     0
Статус                    0
Стаж                      0
На сайте                  0
Заказы в работе           0
Оценка                    0
Количество отзывов        0
Формат обучения           0
Образование              19
Предмет                   0
Цена                      0
С отличием                0
dtype: int64

Это нормально, что в образовании есть немного нанов, далее мы удалим этот признак, остатив другие характеристики, полученные на основе информации о вузе (признаки "топ-10" и "топ-20" вуз).